# Final Project for Course 3 - OMDb and Dad Jokes Mashup

This project will take you through the process of mashing up data from two different APIs.  

[The OMDb API](https://www.omdbapi.com/) lets you provide a movie title as a query input and get back data about the movie, including scores from various review sites (Rotten Tomatoes, IMDb, etc.).

[icanhazdadjokes.com](https://icanhazdadjoke.com/) returns random dad jokes containing a search string that you specify in your query.

The end result of this project will be a function that takes in a movie title as input and produces a formatted text string that includes a couple dad jokes related to a word from the movie's plot.

For example, here are a couple of sample outputs:

---

```
Baby Mama
Rotten Tomatoes rating: 63%
A successful, single businesswoman who dreams of having a baby discovers she is infertile and hires a **working** class woman to be her unlikely surrogate.
Speaking of **working**, that reminds me of some jokes.
Hope they're better than the movie!

Want to hear a joke about construction? Nah, I'm still **working** on it.
Why doesn't the Chimney-Sweep call out sick from work? Because he's used to **working** with a flue.
```

---

```
Back to the Future
Rotten Tomatoes rating: 93%
Marty McFly, a 17-year-old high school student, is **accidentally** sent 30 years into the past in a time-traveling DeLorean invented by his close friend, the maverick scientist Doc Brown.
Speaking of **accidentally**, that reminds me of some jokes.
Hope they're as good as the movie!

I **accidentally** drank a bottle of invisible ink. Now I’m in hospital, waiting to be seen.
A butcher **accidentally** backed into his meat grinder and got a little behind in his work that day.
```
---


To avoid problems with rate limits and site accessibility, we have provided a cache file with results for all the queries you need to make to both OMDb and icanhazdadjokes. Just use `requests_with_caching.get()` rather than `requests.get()`. If you're having trouble, you may not be formatting your queries properly, or you may not be asking for data that exists in our cache. We will try to provide as much information as we can to help guide you to form queries for which data exists in the cache.

## ALERT: All Query Results Should Be Found in the Cache File
If you ever run `requests_with_caching.get()` and it says either of the following, then **something was wrong** with your query:
- new; adding to cache
- found in page-specific cache


In [49]:
import requests_with_caching

In [50]:
apikey = "abcd1234"  # you may *optionally* replace this with your API key.
# Note: you do *not* need an API key to complete this assignment. Every request should be in the cache
requests_with_caching.clear_cache()
# print(list(requests_with_caching.perm_cache().keys()))

# Fetching movie info from OMDb
Your first task will be to fetch data from OMDb. The documentation for the API is at [https://www.omdbapi.com/](https://www.omdbapi.com/)

Define a function called `get_movie_data`. It takes in one parameter which is a string that should represent the title of a movie you want to search. The function should return a dictionary with information about that movie.

Again, use `requests_with_caching.get()`. If you were to use the live OMDP API, you would need to get an API key, as described in the documentation. However **you do not need an API key** to complete this assignment. For the queries on movies that are already in the permanent cache, you won’t need an API key. 

HINT: Be sure to include **only** keys `t` and `r` as query parameters in order to extract data from the cache. If any other parameters are included,  the function will not be able to recognize the data that you're attempting to pull from the cache.

The following movie titles are in the cache:
- Black Panther
- Venom
- Baby Mama
- Sherlock Holmes
- Return of the Jedi
- Back to the Future

In [51]:
# NOTE: the OMDb API uses http:// instead of https://
def get_movie_data(name: str) -> dict:
    """Returns a dictionary of movie information from the OMDb API.

    Parameters
    ----------
    name : str
        The name of the movie to search for.

    Returns
    -------
    dict
        A dictionary of movie information.
    """
    
    baseurl = "http://www.omdbapi.com/"
    params_d = {}
    params_d["t"] = r""+name
#    params_d["apikey"] = apikey
    params_d["r"] = "json"
   
    omdb_resp = requests_with_caching.get(baseurl, params = params_d)
    
    print(omdb_resp.url)
    return omdb_resp.json()

result_omdb = get_movie_data("Black Panther")





found in permanent_cache
http://www.omdbapi.com/?t=Black+Panther&r=json


In [52]:
results = get_movie_data("Black Panther")

assert type(results) == type({})
assert results["Year"] == "2018"

# some other invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# print(get_movie_data("Venom"))
# print(get_movie_data("Baby Mama"))

found in permanent_cache
http://www.omdbapi.com/?t=Black+Panther&r=json


In [53]:
for k, val in results.items():
    print(f"{k}: {val}")

Title: Black Panther
Year: 2018
Rated: PG-13
Released: 16 Feb 2018
Runtime: 134 min
Genre: Action, Adventure, Sci-Fi
Director: Ryan Coogler
Writer: Ryan Coogler, Joe Robert Cole, Stan Lee
Actors: Chadwick Boseman, Michael B. Jordan, Lupita Nyong'o
Plot: T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step forward to lead his people into a new future and must confront a challenger from his country's past.
Language: English, Swahili, Nama, Xhosa, Korean
Country: United States
Awards: Won 3 Oscars. 123 wins & 289 nominations total
Poster: https://m.media-amazon.com/images/M/MV5BMTg1MTY2MjYzNV5BMl5BanBnXkFtZTgwMTc4NTMwNDI@._V1_SX300.jpg
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '96%'}, {'Source': 'Metacritic', 'Value': '88/100'}]
Metascore: 88
imdbRating: 7.3
imdbVotes: 837,583
imdbID: tt1825683
Type: movie
DVD: 02 May 2018
BoxOffice: $700,426,566
Production: N/A
Website: N/A
Response: True


In [54]:
print(type(results['Plot']))

<class 'str'>


In [55]:
chk = results["Ratings"][1]['Value'].split("%")
o = int(chk[0])
print(type(o))

<class 'int'>


## Extract the Rotten Tomatoes Rating for a Movie
Next, you will write a function that extracts the Rotten Tomatoes rating for a movie from the results dictionary as an *integer*. If the movie does not have a Rotten Tomatoes rating, return `-1`.

Fill in the template for the function below

In [56]:
def rt_rating(movie_data: dict) -> int:
    """Returns the Rotten Tomatoes rating from a dictionary of movie information.

    Parameters
    ----------
    movie_data : dict
        A dictionary of movie information.

    Returns
    -------
    int
        The Rotten Tomatoes rating. For example, 75% would be returned as the integer 75.
    """
    rating = movie_data["Ratings"][1]['Value'].split("%")
    return int(rating[0])




# We suggest that you write an assert statement to check the output of your function for the movie Black Panther. The autograder will check results for some other movies.
results = get_movie_data("Black Panther")
assert rt_rating(results) == 96

found in permanent_cache
http://www.omdbapi.com/?t=Black+Panther&r=json


# Fetching Jokes
Now you will use another API to fetch a couple of dad jokes related to a movie's plot.

You will do this in two stages. First you'll implement a helper function that calls the API to get jokes, asking for jokes related to a single word.

Then you'll use that helper function, calling it with the longest words from the plot summary until it finds one that there are jokes for.


## Search for Jokes Containing a Word
To search for dad jokes, you'll be using the API for icanhazdadjokes. The documentation for the API is at [https://icanhazdadjoke.com/api](https://icanhazdadjoke.com/api)

Define a function called `get_joke_data`. It takes in one parameter which is a string. The function should return a dictionary with information about **up to two** jokes that contain that string.

Again, use `requests_with_caching.get()`. All the query results you need are already in the permanent cache.

- Note 1: Be sure to include **only** keys `term` and `limit` as query parameters in order to extract data from the cache. If any other parameters are included, the function will not be able to recognize the data that you're attempting to pull from the cache.
- Note 2: Use the `limit` parameter in the icanhazdadjokes API to limit it to two results (instead of slicing)

In [57]:
import json

In [58]:
# no template is provided for this function. You have to define it from scratch

def get_joke_data(word: str) -> dict:
    """Returns a dictionary of movie information from the OMDb API.

    Parameters
    ----------
    name : str
        The name of the movie to search for.

    Returns
    -------
    dict
        A dictionary of movie information.
    """
    
    baseurl = "https://icanhazdadjoke.com/search"
    params_d = {}
    params_d["term"] = r""+word
#    params_d["apikey"] = apikey
    params_d["limit"] = 2
   
    dadjoke_resp = requests_with_caching.get(baseurl, params = params_d)
    
    print(dadjoke_resp.url)
    
    return dadjoke_resp.json()




In [134]:
res = get_joke_data("challenger")

found in permanent_cache
https://icanhazdadjoke.com/search?term=challenger&limit=2


In [135]:
res

{'current_page': 1,
 'limit': 2,
 'next_page': 1,
 'previous_page': 1,
 'results': [],
 'search_term': 'challenger',
 'status': 200,
 'total_jokes': 0,
 'total_pages': 1}

In [95]:
joke_chk = []
for item in  res["results"]:
    joke_chk.append(item['joke'])
print(joke_chk)
    

['Sometimes I tuck my knees into my chest and lean forward.  That’s just how I roll.', 'Why do scuba divers fall backwards into the water? Because if they fell forwards they’d still be in the boat.']


In [86]:
for k,v in res["results"][0].items():
    print(f"Key {k}: value {v}, {type(v)}")

Key id: value 0189hNRf2g, <class 'str'>
Key joke: value I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later., <class 'str'>


In [87]:
res["results"][0]["joke"]

"I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later."

In [62]:
len(res)

9

In [63]:
assert (
    len(get_joke_data("magic")["results"]) == 2
), "The correct number of jokes for 'magic' should be 2"
assert (
    get_joke_data("magic")["results"][0]["joke"]
    == "What do you call a magician who has lost their magic? Ian."
)

found in permanent_cache
https://icanhazdadjoke.com/search?term=magic&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=magic&limit=2


## Get Jokes for a Long Word from the Plot Description

Now you'll define a function called `get_jokes`. It will extract the longest word from the plot and try to find jokes for it. If there aren't any, it will proceed to the next longest word, and so on, until it finds a word for which there are jokes. If there is more than one word of the same length, try words that are earlier in the description first (which `sorted` does by default, since it's "stable" and will only move things around the minimum necessary).

We provide code that removes punctuation from the words in `plot` and assigns the result to the variable `words`. Your code should extend this by sorting `words` from longest to shortest and use the sorted list (and the `get_joke_data` function that you defined above) to find the longest word with associated jokes. If there are no words with associated jokes, your function should return the tuple `(None, None)`. If there is a word with associated jokes, your function should return a tuple with (1) the longest word with a joke and (2) a list of jokes associated with that word (as a list of strings).

In [158]:
def get_jokes(plot: str, verbosity=0) -> tuple[str, list[str]]:
    """Returns a tuple containing the longest word for which jokes were found
    and the joke itself. Break ties for longest word using the order in `plot`.
    Make sure that you strip punctuation from the word before you search for a joke.

    Parameters
    ----------
    plot : str
        The plot of a movie.

    verbosity : int (optional)
        If 0, no output is printed. If 1, some output is printed about which words were tried.
        Defaults to 0.

    Returns
    -------
    tuple[str, list[str]]
        A tuple containing the word that was used to search for a joke and a list of two joke strings.
    """

    words = plot.split()  # split into separate words
    words = [w.strip(",.!;:") for w in words]  # remove punctuation for each word
    words = sorted(words, key=lambda x: (-len(x),x))
    
    for word in words:
        
        r= get_joke_data(word)
        if len(r["results"])>0:
            print(word)
            joke_ = []
            for item in  r["results"]:
                joke_.append(item['joke'])
            return (word, joke_)
        
#    return words
    
    return (None, None)
    # WRITE YOUR CODE HERE




In [137]:
plot = "T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step forward to lead his people into a new future and must confront a challenger from his country's past."

In [138]:
#plot = "I had dreams of a cat."
result = get_jokes(plot, 1)

found in permanent_cache
https://icanhazdadjoke.com/search?term=challenger&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=country%27s&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=T%27Challa&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=advanced&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=confront&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=Wakanda&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=forward&limit=2
forward


In [139]:
#results[1]

In [140]:
x,y = get_jokes(plot, 1)

found in permanent_cache
https://icanhazdadjoke.com/search?term=challenger&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=country%27s&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=T%27Challa&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=advanced&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=confront&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=Wakanda&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=forward&limit=2
forward


In [141]:
print(x)
print(y)

forward
['Sometimes I tuck my knees into my chest and lean forward.  That’s just how I roll.', 'Why do scuba divers fall backwards into the water? Because if they fell forwards they’d still be in the boat.']


In [142]:
# Test case 1: Plot with a word that has jokes
plot = "I had dreams of a cat."
result = get_jokes(plot, 1)
print(result)
assert result[0] == "dreams"
assert (
    result[1][0]
    == "I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later."
)

found in permanent_cache
https://icanhazdadjoke.com/search?term=dreams&limit=2
dreams
('dreams', ["I'm tired of following my dreams. I'm just going to ask them where they are going and meet up with them later."])


BONUS CHALLENGE (ungraded). If we had specified that ties should be broken by taking words in the alphabetic order rather than later, how could you have done that? Try writing a test and then implementing it!

## Put it All Together

Now put it all together to make the full app. Define a function, `haha_me`. It takes a movie name as input and verbosity and returns a text string that is meant to entertain the reader.

We have provided a helper function, `highlight`, that highlights a string inside a larger string by wrapping it in asterisks (`*`). Try invoking it a few times to make sure you understand what it does, then figure out how it should be used based on the sample outputs in the assert statements.

If the movie is not found in the OMDb API (using `get_movie_data`), return `"No movie found: "` followed by the movie title.

If the movie is found, but there are no jokes (`get_jokes` returned `(None, None)`), return `"I've got no jokes about this movie. It's too serious!"`.

If the movie and jokes are found, your function should return a string with each of the following on separate lines:
- The name of the movie
- `"Rotten Tomatoes rating: XX%"` (replacing `"XX"` with the actual Rotten Tomatoes rating)
- The plot of the movie with the joke keyword highlighted (using the provided `highlight` function)
- `"Speaking of **YY**, that reminds me of some jokes."` (replacing `"YY"` with the joke keyword)
- A different phrase about the jokes, depending on the Rotten Tomatoes rating:
    - No Rotten Tomates Rating (meaning the rating is `-1`): `"Hope you like them!"`
    - Rotten Tomatoes Rating below 70%: `"Hope they're better than the movie!"`
    - Rotten Tomates 70%+: `"Hope they're as good as the movie!"`
- *(an empty line)*
- The list of jokes, separated by a newline (using `"\n".join(...)`)

For example, for Venom:
```
Venom
Rotten Tomatoes rating: 30%
A failed reporter is bonded to an alien entity, one of many symbiotes who have invaded **Earth**. But the being takes a liking to **Earth** and decides to protect it.
Speaking of **Earth**, that reminds me of some jokes.
Hope they're better than the movie!

Astronomers got tired watching the moon go around the **Earth** for 24 hours. They decided to call it a day.
The rotation of **Earth** really makes my day.
```

In [143]:
def highlight(word: str, sentence: str) -> str:
    """
    Highlights a specific word in a sentence by surrounding it with asterisks (**).
    The highlighting is case-insensitive.

    Args:
        word (str): The word to be highlighted.
        sentence (str): The sentence in which the word should be highlighted.

    Returns:
        str: The sentence with the specified word highlighted.
    """
    import re

    # Escape special characters in the word to treat it as a literal string
    # Use re.sub() to replace the word with the highlighted version
    # Use re.IGNORECASE flag to perform case-insensitive replacement
    return re.sub(re.escape(word), f"**{word}**", sentence, flags=re.IGNORECASE)

## Put it All Together

Now put it all together to make the full app. Define a function, `haha_me`. It takes a movie name as input and verbosity and returns a text string that is meant to entertain the reader.

We have provided a helper function, `highlight`, that highlights a string inside a larger string by wrapping it in asterisks (`*`). Try invoking it a few times to make sure you understand what it does, then figure out how it should be used based on the sample outputs in the assert statements.

If the movie is not found in the OMDb API (using `get_movie_data`), return `"No movie found: "` followed by the movie title.

If the movie is found, but there are no jokes (`get_jokes` returned `(None, None)`), return `"I've got no jokes about this movie. It's too serious!"`.

If the movie and jokes are found, your function should return a string with each of the following on separate lines:
- The name of the movie
- `"Rotten Tomatoes rating: XX%"` (replacing `"XX"` with the actual Rotten Tomatoes rating)
- The plot of the movie with the joke keyword highlighted (using the provided `highlight` function)
- `"Speaking of **YY**, that reminds me of some jokes."` (replacing `"YY"` with the joke keyword)
- A different phrase about the jokes, depending on the Rotten Tomatoes rating:
    - No Rotten Tomates Rating (meaning the rating is `-1`): `"Hope you like them!"`
    - Rotten Tomatoes Rating below 70%: `"Hope they're better than the movie!"`
    - Rotten Tomates 70%+: `"Hope they're as good as the movie!"`
- *(an empty line)*
- The list of jokes, separated by a newline (using `"\n".join(...)`)

For example, for Venom:
```
Venom
Rotten Tomatoes rating: 30%
A failed reporter is bonded to an alien entity, one of many symbiotes who have invaded **Earth**. But the being takes a liking to **Earth** and decides to protect it.
Speaking of **Earth**, that reminds me of some jokes.
Hope they're better than the movie!

Astronomers got tired watching the moon go around the **Earth** for 24 hours. They decided to call it a day.
The rotation of **Earth** really makes my day.
```

In [144]:
x =90
y = 1000
chk_msg = f"rating is the {x} '\n' given by {y} people"
print(chk_msg)

rating is the 90 '
' given by 1000 people


In [145]:
lst1 = ["hi-----", "___there"]
print("\n".join(lst1))


hi-----
___there


In [146]:

movie_result = get_movie_data("Black Panther")

found in permanent_cache
http://www.omdbapi.com/?t=Black+Panther&r=json


In [147]:
plot = movie_result['Plot']
print(plot)

T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step forward to lead his people into a new future and must confront a challenger from his country's past.


In [148]:
word, jokes = get_jokes(plot)

found in permanent_cache
https://icanhazdadjoke.com/search?term=challenger&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=country%27s&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=T%27Challa&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=advanced&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=confront&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=Wakanda&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=forward&limit=2
forward


In [149]:
jokes

['Sometimes I tuck my knees into my chest and lean forward.  That’s just how I roll.',
 'Why do scuba divers fall backwards into the water? Because if they fell forwards they’d still be in the boat.']

In [155]:
def haha_me(movie_title: str, verbosity=0) -> str:
    """Returns a string containing a movie title, the Rotten Tomatoes rating, a plot summary,
    and two jokes about the movie. If no jokes are found, a message is returned instead.

    Parameters
    ----------
    movie_title : str
        The title of a movie.

    verbosity : int (optional)
        If 0, no output is printed. If 1, some output is printed about which plot words were tried.
        Defaults to 0.
    """
    movie_result = get_movie_data(movie_title)
    if len(movie_result)==0:
        message = f" No movie found: {movie_title}"
        return message
    
    plot = movie_result['Plot']
    word, jokes = get_jokes(plot)
    
    if len(jokes)==0:
        message = "I've got no jokes about this movie. It's too serious!"
        return message
    
    rating = rt_rating(movie_result)
    highlighted_msg = highlight(word, plot)
    if rating>=70:
        phrase = "Hope they're as good as the movie!"
    elif rating<70:
        phrase = "Hope they're better than the movie!"
    else:
        phrase = "Hope you like them!"
        
    msg1 = movie_title
    msg2 = "Rotten Tomatoes rating: "+str(rating)+"%"
    msg3 = highlighted_msg
    msg4 = "Speaking of **"+word+"**, that reminds me of some jokes."
    msg5 = phrase
    msg6 = "\n"
    msg7 = "\n".join(jokes)
    msg7 = highlight(word, msg7)
    final = msg1+"\n"+msg2+"\n"+msg3+"\n"+msg4+"\n"+msg5+"\n"+msg6+msg7
    return final
    
        
        
    




In [156]:
print(haha_me("Black Panther"))

found in permanent_cache
http://www.omdbapi.com/?t=Black+Panther&r=json
found in permanent_cache
https://icanhazdadjoke.com/search?term=challenger&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=country%27s&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=T%27Challa&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=advanced&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=confront&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=Wakanda&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=forward&limit=2
forward
Black Panther
Rotten Tomatoes rating: 96%
T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step **forward** to lead his people into a new future and must confront a challenger from his country's past.
Speaking of **forward**, that reminds me of some jokes.
Hope they're as good as the movie!

Sometimes I tuck my knees into my che

In [157]:
assert (
    haha_me("Black Panther")
    == """Black Panther
Rotten Tomatoes rating: 96%
T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step **forward** to lead his people into a new future and must confront a challenger from his country's past.
Speaking of **forward**, that reminds me of some jokes.
Hope they're as good as the movie!

Sometimes I tuck my knees into my chest and lean **forward**.  That’s just how I roll.
Why do scuba divers fall backwards into the water? Because if they fell **forward**s they’d still be in the boat."""
)

found in permanent_cache
http://www.omdbapi.com/?t=Black+Panther&r=json
found in permanent_cache
https://icanhazdadjoke.com/search?term=challenger&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=country%27s&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=T%27Challa&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=advanced&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=confront&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=Wakanda&limit=2
found in permanent_cache
https://icanhazdadjoke.com/search?term=forward&limit=2
forward
